# Spectral Theory 

Spectral theory describes how to decouple the domain of a linear operator into 
a direct sum of minimal components upon which the operator is invariant. 
Choosing a basis that respects this direct sum results in a corresponding 
block-diagonal matrix representation. This has powerful consequences in 
applications, as it allows many problems to be reduced to a series of 
small individual parts that can be solved independently and more easily. 
The key tools for constructing this decomposition are eigenvalues and 
eigenvectors, which are widely used in many areas of science and engineering 
and have many important physical interpretations and applications.

Most finite-dimensional linear operators have eigenvectors that span the 
space (hereafter called an eigenbasis) where the corresponding matrix 
representation is diagonal. In other words, a change of basis to the 
eigenbasis shows that such a matrix operator is similar to a diagonal matrix. 
That said, not all matrices can be diagonalized, and some can only be made 
block diagonal. In the first sections of this chapter, we develop this 
theory and expound on when a matrix can be diagonalized and when we 
must settle for block diagonalization.

## Eigenvalues and Eigenvectors 


Let $L: V \to V$ be a linear operator on a finite-dimensional vector space
$V$ over $\mathbb{C}$. A scalar $\lambda \in \mathbb{C}$ is an eigenvalue of 
$L$ if there exists a nonzero $x \in V$ such that 

$$L(x)  = \lambda x $$

Any nonzero $x$ satisfying the above equation is called an eigenvector of 
$L$ corresponding to the eigenvalue $\lambda$. For each scalar $\lambda$, 
we define the $\lambda$-eigenspace of $L$ as 

$$\sum_{\lambda} (L) = \{ x \in V | L(x) = \lambda x \} $$

The set of all eigenvalues of $L$, denoted $\sigma(L)$, is called the spectrum 
of $L$. 

Let $A \in M_n(\mathbb{F})$ and $\lambda \mathbb{C}$. The following are 
equivalent:

- $\lambda$ is an eigenvalue of $A$ 
- There is a nonzero $x$ such that $(\lambda I - A)x = 0$ 
- $\lambda I - A$ is singular 
- det($\lambda I - A$) = 0

Let $A \in M_n(\mathbb{F})$. The polynomial 

$$p_A(z) = \text{det} (zI - A)$$

is called the characteristic polynomial of $A$. 

A square matrix $A$ is said to be similar to another square matrix $B$ if and 
only if there exists an invertible matrix $P$ such that 

$$B = P^{-1} A P$$

The matrix $P$ is called change-of-basis matrix. 

If $A, B \in M_n(\mathbb{F})$ are similar matrices, that is, $B = P^{-1}AP$
for some nonsingular matrix $P$, then the following hold:

- $A$ and $B$ have the same characteristic polynomials
- $A$ and $B$ have the same eigenvalues 

An operator $L$ on a finite-dimensional vector space is called

- simple if all of its eigenvalues are distinct
- semisimple if there exists an eigenbasis of $L$

The matrix $A$ is diagonalizable if it is similar to a diagonal matrix;
that is, there exists a nonsingular matrix $P$ and a diagonal matrix $D$ 
such that 

$$D = P^{-1}AP$$

A matrix is diagonalizable if and only if it is semisimple. 

## Schur's Lemma 


Schur's lemma states that any square matrix can be transformed via similarity 
to an upper-triangular matrix and that the similarity transform can be 
performed with an orthonormal matrix. At first glance, this may seem unimportant, 
but Schur's lemma is quite powerful, both theoretically and computationally.